## Install All the Required Packages

In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf

In [ ]:
!pip install openai
!pip install tiktoken

## Import All the Required Libraries

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

## Load the PDF Files

In [ ]:
!mkdir pdfs

## Extract the Text from the PDF's

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [ ]:
data

## Split the Extracted Data into Text Chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
text_chunks

In [ ]:
len(text_chunks)

In [ ]:
text_chunks[1]

## Download the Embeddings
### Else you can use hugging face Embedding modle like "mpnet v2"

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = "sush-xxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
result = embeddings.embed_query("Are you doing good")

In [ ]:
len(result)

## Initializing the Pinecone

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 're57ft6e12r-5re12r')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [ ]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "test" # put in the name of your pinecone index here


## Create Embeddings for each of the Text Chunk

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

## If you already have an index, you can load it like this

In [ ]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)
docsearch

## Similarity Search

In [ ]:
query = "query what to search"

In [ ]:
docs = docsearch.similarity_search(query, k=3)

In [ ]:
docs

## Creating a LLM Model Wrapper

In [ ]:
llm = OpenAI()

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


## Q/A

In [ ]:
query = "query what to search2"

In [ ]:
qa.run(query)

In [ ]:
query = "query what to search3"

In [ ]:
qa.run(query)

In [ ]:
import sys

In [ ]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")